# 債券報價中的 YTM、Spot Rate、Forward Rate

## 輸入資料：債券現值、面值、年期、配息、付息次數

In [1]:
pre_value = float(input("Input the bond's present value:(請輸入債券現值):"))  #債券現值、價格

Input the bond's present value:(請輸入債券現值):99


In [2]:
par_value = float(input("Input the bond's par value:(請輸入債券面值)"))  #債券票面價值

Input the bond's par value:(請輸入債券面值)100


In [3]:
year = int(input("Input the year of the bond:(請輸入債券的年期):"))  #年期數

Input the year of the bond:(請輸入債券的年期):2


In [4]:
bond_coup = float(input("Input the coupon rate(%) of the bond :(請輸入債息百分比):"))  #債券配息率

Input the coupon rate(%) of the bond :(請輸入債息百分比):2


In [5]:
freq = int(input("Input the frequency of interest payments per year:"))  #每年付息次數

Input the frequency of interest payments per year:2


## 計算債券到期收益率(YTM)的函數

In [43]:
def is_pos_num(s):
    try:
        int(s)
    except:
        return False
    if s<=0:
        return False
    return True

In [52]:
from sympy import *
import scipy.optimize as optimize
def YTM_of_bond(pre, par, year, coup, freq):
    periods = year*freq
    per_coup = coup/100*par/freq
    
    y = symbols('y')
    func = sum([per_coup/(1+y/freq)**(i+1) for i in range(periods)]) + par/(1+y/freq)**periods - pre
    
    #return solve(func,y)
    return [x for x in solve(func,y) if is_pos_num(x)]

In [53]:
ytm = YTM_of_bond(pre_value, par_value, year, bond_coup, freq)[0]
ytm

0.0251582216359343

In [ ]:
## 計算Spot Rate